In [ ]:
from GraphModule.gcn import MyGCN
from GraphModule.nettack import Nettack

In [ ]:
import random
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
from scipy import sparse

import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F

from deeprobust.graph.defense import GCN
#from deeprobust.graph.targeted_attack import Nettack
from deeprobust.graph.utils import accuracy
from deeprobust.graph.data import Dataset

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
torch.manual_seed(0)
random.seed(0)
np.random.seed(0)

def test_MyGCN(target_node, ori_adj, aug_adj, features, labels, train_idx, val_idx, test_idx, lamb=1, verbose=True):
    ''' test on GCN '''
    my_gcn = MyGCN(nfeat=features.shape[1], nhid=16, nclass=labels.max().item()+1, lamb=lamb, dropout=0.5, device=device)
    my_gcn = my_gcn.to(device)

    my_gcn.fit(features, ori_adj, aug_adj, labels, train_idx, val_idx, patience=30)

    my_gcn.eval()
    output = my_gcn.predict()
    probs = torch.exp(output[[target_node]])[0]
    acc_test = accuracy(output[test_idx], labels[test_idx])
    if verbose:
        print('Target node probs: {}'.format(probs.detach().cpu().numpy().round(decimals=5)))
        print('Target node classification: {}'.format(probs.detach().cpu().numpy().argmax()))
        print('Target node labels: {}'.format(labels[target_node]))
        print("Overall test set results:", "accuracy= {:.4f}".format(acc_test.item()))
    #print(type(output[[target_node]].detach().cpu().numpy()))
    return output[[target_node]][0].detach().cpu().numpy()

#test_MyGCN(target_node, adj, arg_adj, features, labels, idx_train, idx_value, idx_test, lamb=1)

In [ ]:
data = Dataset(root='Dataset', name='cora', seed=15)
adj, features, labels = data.adj, data.features, data.labels
idx_train, idx_value, idx_test = data.idx_train, data.idx_val, data.idx_test

idx_unlabeled = np.union1d(idx_value, idx_test)
idx_used = np.union1d(idx_train, idx_unlabeled)
target_node = 0
assert target_node in idx_unlabeled

In [ ]:
#defense_nodes = [10, 20, 30, 40, 50, 60, 70, 80, 90, 100] #random choose
#defense_nodes = [1525, 389, 1529, 1808, 1602, 442, 1115, 1141, 2019, 1006] # training set
defense_nodes = [381, 894, 518, 43, 2082, 2332, 851, 216, 424] # cheat

def argument_graph(target_node):  
    arg_adj = adj.todense()
    for node in defense_nodes[:5]:
        arg_adj[target_node,node] = abs( 1 - arg_adj[target_node,node])
    return sparse.csr_matrix(arg_adj)
arg_adj = argument_graph(target_node)

In [ ]:
def test_nettack_with_adj(target_node, test_adj, test_model):
    nettack = Nettack(test_model, nnodes=adj.shape[0], attack_structure=True, attack_features=False, device=device)
    nettack = nettack.to(device)
    n_perturbations = 5

    nettack.attack(features, test_adj, labels, target_node, n_perturbations,verbose=False)
    return [ i[1] for i in nettack.structure_perturbations]

In [ ]:
test_MyGCN(target_node, adj, arg_adj, features, labels, idx_train, idx_value, idx_test, lamb=1)

In [ ]:
mygcn = MyGCN(nfeat=features.shape[1], nhid=16, nclass=labels.max().item()+1, lamb=0, dropout=0.5, device=device)
mygcn.fit(features, adj, arg_adj, labels, idx_train, idx_value)

In [ ]:
nettack = Nettack(mygcn, nnodes=adj.shape[0], attack_structure=True, attack_features=False, device=device)
nettack = nettack.to(device)

In [ ]:
degrees = adj.sum(0).A1
n_perturbations = 5

nettack.attack(features, adj, labels, target_node, n_perturbations, verbose=False)
print([ i[1] for i in nettack.structure_perturbations ] )

In [ ]:
modified_adj = nettack.modified_adj
modified_features = nettack.modified_features

In [ ]:
a = test_MyGCN(target_node, adj, arg_adj,  features, labels, idx_train, idx_value, idx_test)
print()
b = test_MyGCN(target_node, arg_adj, arg_adj,  features, labels, idx_train, idx_value, idx_test)
print()
c = test_MyGCN(target_node, modified_adj, arg_adj,  features, labels, idx_train, idx_value, idx_test)
print()

from sklearn.metrics.pairwise import cosine_similarity
print("a b cosine similarity: {:5f}".format(cosine_similarity([a], [b])[0,0]))
print("a c cosine similarity: {:5f}".format(cosine_similarity([a], [c])[0,0]))
print("b c cosine similarity: {:5f}".format(cosine_similarity([b], [c])[0,0]))

In [ ]:
a1 = test_MyGCN(target_node, adj, arg_adj,  features, labels, idx_train, idx_value, idx_test, verbose=False)
a2 = test_MyGCN(target_node, adj, arg_adj,  features, labels, idx_train, idx_value, idx_test, verbose=False)

b1 = test_MyGCN(target_node, arg_adj, arg_adj,  features, labels, idx_train, idx_value, idx_test, verbose=False)
b2 = test_MyGCN(target_node, arg_adj, arg_adj,  features, labels, idx_train, idx_value, idx_test, verbose=False)

c1 = test_MyGCN(target_node, modified_adj, arg_adj,  features, labels, idx_train, idx_value, idx_test, verbose=False)
c2 = test_MyGCN(target_node, modified_adj, arg_adj,  features, labels, idx_train, idx_value, idx_test, verbose=False)

from sklearn.metrics.pairwise import cosine_similarity
print("a a cosine similarity: {:5f}".format(cosine_similarity([a1], [a2])[0,0]))
print("b b cosine similarity: {:5f}".format(cosine_similarity([b1], [b2])[0,0]))
print("c c cosine similarity: {:5f}".format(cosine_similarity([c1], [c2])[0,0]))